In [1]:
import pandas as pd 
import catboost as cb 
from sklearn.model_selection import train_test_split

In [2]:
train_values = pd.read_csv("train_values_catboost.csv", index_col='building_id')
train_labels = pd.read_csv('train_labels.csv', index_col='building_id')

In [33]:
test_values =  pd.read_csv("test_values.csv", index_col='building_id')
test_values1 =  pd.read_csv("Archivos auxiliares KNN/test_values_short1.csv", index_col='building_id')

In [3]:
importances_superestructures = [0.11087213, 0.24814224, 0.06244005, 0.03015171, 0.08679662,
       0.1561809 , 0.0585955 , 0.04264239, 0.07171384, 0.11541968,
       0.01704496]

superestructures = ['has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
       'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other']

reemplazos = {}
for i in range(len(superestructures)):
    reemplazos[superestructures[i]] = {1:importances_superestructures[i]}    

In [4]:
test_values.replace(reemplazos, inplace=True)

In [6]:
test_values['hasnt_secondary'] = (test_values['has_secondary_use'] != True) * 1

In [7]:
importances_secondary_uses = [0.19752775, 0.27044932, 0.21866441, 0.02404361, 0.00476977,
       0.00504387, 0.00198575, 0.00234236, 0.00102254, 0.02242508,
       0.25172554]
secondary_uses =  ['has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_health_post', 'has_secondary_use_gov_office',
       'has_secondary_use_use_police', 'has_secondary_use_other', 'hasnt_secondary']

reemplazos = {}
for i in range(len(secondary_uses)):
    reemplazos[secondary_uses[i]] = {1:importances_secondary_uses[i]}    

In [8]:
test_values.replace(reemplazos, inplace=True)

In [10]:
test_values ['superestructure'] = test_values\
                .loc[:, superestructures].idxmax(axis='columns')
test_values.drop(columns=superestructures, inplace=True)


test_values ['secondary_use'] = test_values\
                .loc[:, secondary_uses].idxmax(axis='columns')
test_values.drop(columns=secondary_uses, inplace=True)

In [11]:
test_values.drop(columns=["has_secondary_use"], inplace=True)
test_values.drop(columns=[ "geo_level_1_id","geo_level_2_id","geo_level_3_id"], inplace=True)

In [14]:
test_values = test_values.merge(test_values1[["min_ic", "max_ic"]],right_index=True, left_index=True)

In [15]:
test_values.to_csv('test_values_catboost.csv', index=True)

In [3]:
# A excepcion de los geo_levels todas las columnas tienen pocos valores por lo que 
# se usar todas como categoricas
categoricas = []
for col in train_values.columns:
    if ((col != 'min_geo_id') & (col != 'max_geo_id')):
        categoricas.append(col)
        categoricas.append(train_values.columns.get_loc(col))
        train_values[col] = train_values[col].astype("category")

In [4]:
model = cb.CatBoostClassifier(loss_function='MultiClass', eval_metric='TotalF1:average=Micro', iterations=2000)

In [5]:
model.fit(train_values, train_labels, categoricas)

Learning rate set to 0.058502
0:	learn: 0.7180287	total: 1.4s	remaining: 46m 33s
1:	learn: 0.7302466	total: 2.65s	remaining: 44m 2s
2:	learn: 0.7299281	total: 3.85s	remaining: 42m 42s
3:	learn: 0.7334431	total: 5.03s	remaining: 41m 49s
4:	learn: 0.7348283	total: 6.19s	remaining: 41m 10s
5:	learn: 0.7361484	total: 7.42s	remaining: 41m 7s
6:	learn: 0.7358759	total: 8.6s	remaining: 40m 48s
7:	learn: 0.7352044	total: 9.86s	remaining: 40m 55s
8:	learn: 0.7354500	total: 11.1s	remaining: 40m 53s
9:	learn: 0.7353732	total: 12.3s	remaining: 40m 44s
10:	learn: 0.7368736	total: 13.5s	remaining: 40m 37s
11:	learn: 0.7374147	total: 14.7s	remaining: 40m 38s
12:	learn: 0.7385966	total: 15.9s	remaining: 40m 36s
13:	learn: 0.7383164	total: 17.1s	remaining: 40m 25s
14:	learn: 0.7389573	total: 18.3s	remaining: 40m 24s
15:	learn: 0.7392566	total: 19.6s	remaining: 40m 30s
16:	learn: 0.7395022	total: 20.8s	remaining: 40m 28s
17:	learn: 0.7399434	total: 22.1s	remaining: 40m 35s
18:	learn: 0.7400240	total: 23

153:	learn: 0.7538421	total: 3m 16s	remaining: 39m 16s
154:	learn: 0.7538229	total: 3m 18s	remaining: 39m 19s
155:	learn: 0.7539035	total: 3m 19s	remaining: 39m 20s
156:	learn: 0.7539495	total: 3m 21s	remaining: 39m 23s
157:	learn: 0.7539227	total: 3m 22s	remaining: 39m 25s
158:	learn: 0.7539418	total: 3m 24s	remaining: 39m 26s
159:	learn: 0.7539956	total: 3m 25s	remaining: 39m 26s
160:	learn: 0.7540570	total: 3m 27s	remaining: 39m 29s
161:	learn: 0.7540953	total: 3m 28s	remaining: 39m 30s
162:	learn: 0.7540800	total: 3m 30s	remaining: 39m 31s
163:	learn: 0.7540762	total: 3m 31s	remaining: 39m 32s
164:	learn: 0.7541222	total: 3m 33s	remaining: 39m 34s
165:	learn: 0.7542488	total: 3m 35s	remaining: 39m 37s
166:	learn: 0.7543640	total: 3m 36s	remaining: 39m 38s
167:	learn: 0.7543409	total: 3m 38s	remaining: 39m 41s
168:	learn: 0.7544138	total: 3m 40s	remaining: 39m 46s
169:	learn: 0.7544253	total: 3m 41s	remaining: 39m 45s
170:	learn: 0.7544522	total: 3m 43s	remaining: 39m 50s
171:	learn

303:	learn: 0.7566778	total: 6m 44s	remaining: 37m 38s
304:	learn: 0.7566817	total: 6m 46s	remaining: 37m 37s
305:	learn: 0.7566663	total: 6m 47s	remaining: 37m 35s
306:	learn: 0.7566855	total: 6m 48s	remaining: 37m 33s
307:	learn: 0.7567047	total: 6m 50s	remaining: 37m 32s
308:	learn: 0.7566932	total: 6m 51s	remaining: 37m 30s
309:	learn: 0.7566817	total: 6m 52s	remaining: 37m 28s
310:	learn: 0.7567200	total: 6m 53s	remaining: 37m 26s
311:	learn: 0.7568160	total: 6m 55s	remaining: 37m 26s
312:	learn: 0.7568045	total: 6m 56s	remaining: 37m 24s
313:	learn: 0.7568467	total: 6m 57s	remaining: 37m 22s
314:	learn: 0.7568467	total: 6m 59s	remaining: 37m 21s
315:	learn: 0.7568428	total: 7m	remaining: 37m 20s
316:	learn: 0.7568467	total: 7m 1s	remaining: 37m 17s
317:	learn: 0.7568236	total: 7m 2s	remaining: 37m 15s
318:	learn: 0.7568121	total: 7m 4s	remaining: 37m 14s
319:	learn: 0.7568313	total: 7m 5s	remaining: 37m 14s
320:	learn: 0.7568428	total: 7m 6s	remaining: 37m 13s
321:	learn: 0.75688

453:	learn: 0.7579134	total: 10m 9s	remaining: 34m 35s
454:	learn: 0.7579211	total: 10m 10s	remaining: 34m 33s
455:	learn: 0.7579441	total: 10m 11s	remaining: 34m 31s
456:	learn: 0.7579441	total: 10m 13s	remaining: 34m 30s
457:	learn: 0.7579326	total: 10m 14s	remaining: 34m 29s
458:	learn: 0.7579633	total: 10m 16s	remaining: 34m 28s
459:	learn: 0.7579595	total: 10m 17s	remaining: 34m 27s
460:	learn: 0.7579787	total: 10m 18s	remaining: 34m 26s
461:	learn: 0.7579787	total: 10m 20s	remaining: 34m 24s
462:	learn: 0.7579672	total: 10m 21s	remaining: 34m 24s
463:	learn: 0.7580247	total: 10m 23s	remaining: 34m 22s
464:	learn: 0.7580631	total: 10m 24s	remaining: 34m 21s
465:	learn: 0.7580286	total: 10m 25s	remaining: 34m 20s
466:	learn: 0.7580362	total: 10m 27s	remaining: 34m 18s
467:	learn: 0.7580094	total: 10m 28s	remaining: 34m 17s
468:	learn: 0.7580209	total: 10m 29s	remaining: 34m 15s
469:	learn: 0.7580209	total: 10m 31s	remaining: 34m 14s
470:	learn: 0.7579825	total: 10m 32s	remaining: 3

601:	learn: 0.7589150	total: 13m 32s	remaining: 31m 27s
602:	learn: 0.7588919	total: 13m 34s	remaining: 31m 26s
603:	learn: 0.7589226	total: 13m 35s	remaining: 31m 25s
604:	learn: 0.7589265	total: 13m 36s	remaining: 31m 23s
605:	learn: 0.7589649	total: 13m 38s	remaining: 31m 22s
606:	learn: 0.7589994	total: 13m 39s	remaining: 31m 21s
607:	learn: 0.7590263	total: 13m 41s	remaining: 31m 19s
608:	learn: 0.7590032	total: 13m 42s	remaining: 31m 18s
609:	learn: 0.7590224	total: 13m 43s	remaining: 31m 17s
610:	learn: 0.7590071	total: 13m 45s	remaining: 31m 16s
611:	learn: 0.7590109	total: 13m 46s	remaining: 31m 15s
612:	learn: 0.7589917	total: 13m 48s	remaining: 31m 13s
613:	learn: 0.7590032	total: 13m 49s	remaining: 31m 12s
614:	learn: 0.7590032	total: 13m 50s	remaining: 31m 11s
615:	learn: 0.7590224	total: 13m 52s	remaining: 31m 10s
616:	learn: 0.7590301	total: 13m 53s	remaining: 31m 9s
617:	learn: 0.7590569	total: 13m 55s	remaining: 31m 7s
618:	learn: 0.7591222	total: 13m 56s	remaining: 31

...

1335:	learn: 0.7629671	total: 30m 36s	remaining: 15m 12s
1336:	learn: 0.7629518	total: 30m 37s	remaining: 15m 11s
1337:	learn: 0.7629441	total: 30m 38s	remaining: 15m 9s
1338:	learn: 0.7629748	total: 30m 40s	remaining: 15m 8s
1339:	learn: 0.7629978	total: 30m 42s	remaining: 15m 7s
1340:	learn: 0.7630247	total: 30m 43s	remaining: 15m 6s
1341:	learn: 0.7629787	total: 30m 45s	remaining: 15m 4s
1342:	learn: 0.7629825	total: 30m 46s	remaining: 15m 3s
1343:	learn: 0.7629940	total: 30m 48s	remaining: 15m 2s
1344:	learn: 0.7629902	total: 30m 49s	remaining: 15m
1345:	learn: 0.7629940	total: 30m 50s	remaining: 14m 59s
1346:	learn: 0.7629825	total: 30m 51s	remaining: 14m 57s
1347:	learn: 0.7629863	total: 30m 53s	remaining: 14m 56s
1348:	learn: 0.7630055	total: 30m 54s	remaining: 14m 55s
1349:	learn: 0.7630324	total: 30m 56s	remaining: 14m 53s
1350:	learn: 0.7630439	total: 30m 57s	remaining: 14m 52s
1351:	learn: 0.7630400	total: 30m 59s	remaining: 14m 51s
1352:	learn: 0.7629902	total: 31m	remainin

1481:	learn: 0.7635082	total: 34m 2s	remaining: 11m 54s
1482:	learn: 0.7635235	total: 34m 4s	remaining: 11m 52s
1483:	learn: 0.7635542	total: 34m 5s	remaining: 11m 51s
1484:	learn: 0.7635466	total: 34m 7s	remaining: 11m 49s
1485:	learn: 0.7635504	total: 34m 8s	remaining: 11m 48s
1486:	learn: 0.7635581	total: 34m 10s	remaining: 11m 47s
1487:	learn: 0.7635504	total: 34m 12s	remaining: 11m 46s
1488:	learn: 0.7635581	total: 34m 14s	remaining: 11m 45s
1489:	learn: 0.7635504	total: 34m 17s	remaining: 11m 44s
1490:	learn: 0.7635466	total: 34m 19s	remaining: 11m 42s
1491:	learn: 0.7635466	total: 34m 21s	remaining: 11m 41s
1492:	learn: 0.7635274	total: 34m 24s	remaining: 11m 41s
1493:	learn: 0.7635504	total: 34m 26s	remaining: 11m 39s
1494:	learn: 0.7635466	total: 34m 29s	remaining: 11m 38s
1495:	learn: 0.7635926	total: 34m 31s	remaining: 11m 37s
1496:	learn: 0.7636156	total: 34m 33s	remaining: 11m 36s
1497:	learn: 0.7636080	total: 34m 35s	remaining: 11m 35s
1498:	learn: 0.7636348	total: 34m 38

1627:	learn: 0.7643179	total: 39m 36s	remaining: 9m 2s
1628:	learn: 0.7642948	total: 39m 38s	remaining: 9m 1s
1629:	learn: 0.7642948	total: 39m 40s	remaining: 9m
1630:	learn: 0.7642948	total: 39m 42s	remaining: 8m 59s
1631:	learn: 0.7643409	total: 39m 45s	remaining: 8m 57s
1632:	learn: 0.7643793	total: 39m 47s	remaining: 8m 56s
1633:	learn: 0.7643793	total: 39m 49s	remaining: 8m 55s
1634:	learn: 0.7643716	total: 39m 51s	remaining: 8m 53s
1635:	learn: 0.7643754	total: 39m 54s	remaining: 8m 52s
1636:	learn: 0.7643984	total: 39m 56s	remaining: 8m 51s
1637:	learn: 0.7643984	total: 39m 58s	remaining: 8m 50s
1638:	learn: 0.7644176	total: 40m 1s	remaining: 8m 48s
1639:	learn: 0.7643677	total: 40m 3s	remaining: 8m 47s
1640:	learn: 0.7643946	total: 40m 6s	remaining: 8m 46s
1641:	learn: 0.7644023	total: 40m 8s	remaining: 8m 45s
1642:	learn: 0.7644330	total: 40m 10s	remaining: 8m 43s
1643:	learn: 0.7644752	total: 40m 12s	remaining: 8m 42s
1644:	learn: 0.7644982	total: 40m 15s	remaining: 8m 41s
16

1775:	learn: 0.7651122	total: 45m 23s	remaining: 5m 43s
1776:	learn: 0.7651352	total: 45m 25s	remaining: 5m 42s
1777:	learn: 0.7651352	total: 45m 28s	remaining: 5m 40s
1778:	learn: 0.7651122	total: 45m 30s	remaining: 5m 39s
1779:	learn: 0.7651199	total: 45m 33s	remaining: 5m 37s
1780:	learn: 0.7651237	total: 45m 35s	remaining: 5m 36s
1781:	learn: 0.7651199	total: 45m 37s	remaining: 5m 34s
1782:	learn: 0.7651083	total: 45m 39s	remaining: 5m 33s
1783:	learn: 0.7651122	total: 45m 42s	remaining: 5m 32s
1784:	learn: 0.7651275	total: 45m 44s	remaining: 5m 30s
1785:	learn: 0.7651160	total: 45m 47s	remaining: 5m 29s
1786:	learn: 0.7650853	total: 45m 49s	remaining: 5m 27s
1787:	learn: 0.7651237	total: 45m 51s	remaining: 5m 26s
1788:	learn: 0.7651160	total: 45m 53s	remaining: 5m 24s
1789:	learn: 0.7651697	total: 45m 56s	remaining: 5m 23s
1790:	learn: 0.7651697	total: 45m 58s	remaining: 5m 21s
1791:	learn: 0.7651889	total: 46m 1s	remaining: 5m 20s
1792:	learn: 0.7651582	total: 46m 3s	remaining: 5

1923:	learn: 0.7657722	total: 49m 46s	remaining: 1m 57s
1924:	learn: 0.7657799	total: 49m 48s	remaining: 1m 56s
1925:	learn: 0.7657453	total: 49m 49s	remaining: 1m 54s
1926:	learn: 0.7657223	total: 49m 51s	remaining: 1m 53s
1927:	learn: 0.7657453	total: 49m 52s	remaining: 1m 51s
1928:	learn: 0.7657377	total: 49m 54s	remaining: 1m 50s
1929:	learn: 0.7657453	total: 49m 55s	remaining: 1m 48s
1930:	learn: 0.7657914	total: 49m 57s	remaining: 1m 47s
1931:	learn: 0.7658029	total: 49m 58s	remaining: 1m 45s
1932:	learn: 0.7658144	total: 49m 59s	remaining: 1m 43s
1933:	learn: 0.7658106	total: 50m 1s	remaining: 1m 42s
1934:	learn: 0.7658221	total: 50m 3s	remaining: 1m 40s
1935:	learn: 0.7658221	total: 50m 4s	remaining: 1m 39s
1936:	learn: 0.7658298	total: 50m 5s	remaining: 1m 37s
1937:	learn: 0.7658144	total: 50m 7s	remaining: 1m 36s
1938:	learn: 0.7658221	total: 50m 9s	remaining: 1m 34s
1939:	learn: 0.7658182	total: 50m 10s	remaining: 1m 33s
1940:	learn: 0.7658106	total: 50m 12s	remaining: 1m 31

In [6]:
test_values = pd.read_csv("test_values_catboost.csv", index_col='building_id')

In [7]:
pred = model.predict(test_values) 

In [8]:
submission_format = pd.read_csv('submission_format.csv', index_col='building_id')
submission = pd.DataFrame(data=pred,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [10]:
submission.head()

,damage_grade
building_id,
300051,3
99355,2
890251,3
745817,1
421793,3


In [11]:
submission.to_csv('subs/catboost_2000_iterations.csv',index=True)

In [13]:
probas = model.predict_proba(test_values)

In [14]:
probas

array([[2.89774232e-03, 1.70795349e-01, 8.26306908e-01],
       [5.13268601e-04, 9.96017290e-01, 3.46944137e-03],
       [8.46309769e-03, 4.06953283e-02, 9.50841574e-01],
       ...,
       [5.08404018e-02, 9.06344107e-01, 4.28154914e-02],
       [3.07750954e-02, 7.25005482e-01, 2.44219423e-01],
       [5.97895375e-01, 3.32964915e-01, 6.91397103e-02]])

In [15]:
probas_df = pd.DataFrame(data=probas,
                             columns=[1,2,3],
                             index=test_values.index)

In [18]:
probas_df.to_csv('predict_probas/catboost_2000_iterations.csv', index=True)